<div dir="rtl">
    <h2>
       فاز سوم پروژه بازیابی اطلاعات - خوشه‌بندی
    </h2>
</div>
<p></p>
<div dir="rtl">
اعضای گروه: حمیدرضا هدایتی، حامد علی‌محمدزاده، آرمین سعادت‌بروجنی
<p></p>
</div>

<div dir="rtl">
    <h3>
       نصب نیازمندی‌های پروژه
    </h3>
</div>

In [1]:
%cd ../../
%ls

/home/rmool/MIR
classifiers/  data/          IR_files/    __pycache__/      search/
clusterings/  file_handler/  main.py      README.md         venv/
compressor/   helper.py      pagerank/    reports/
crawler/      index/         preprocess/  requirements.txt


In [2]:
!pip install -r requirements.txt
!pip install yellowbrick

<div dir="rtl">
    <h3>
        ایمپورت کردن پکیج‌های مورد نیاز
    </h3>
</div>

In [3]:
from main import JSON_to_clustering_arrays 
from yellowbrick.cluster import intercluster_distance, kelbow_visualizer
from clusterings import k_means
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to /home/rmool/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/rmool/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


<div dir="rtl">
    <h3>
       تحلیل اطلاعات فایل جیسون و ایجاد w2v و tf_idf
    </h3>
<p></p>    
در این قسمت فایل جیسون خوانده شده و اطلاعاتی نظیر لینک و تگ‌های اصلی مستقیما ذخیره می‌شوند. از آن‌جا که ۱۴ دسته‌بندی اصلی در داده‌ها وجود دارد، به جای تگ‌های اصلی اعداد ۰ تا ۱۳ اختصاص داده می‌شود.
<p></p>
قسمت summery و title با هم ترکیب شده و با استفاده از کتابخانه‌های موجود به فضای w2v و tf-idf برده می‌شوند.
<p></p>
    مفهموم فاصله که مبنای کار k_means است در ابعاد بالا دچار مشکل می‌شود. بنابراین با استفاده از PCA فضای برداری را به فضای با بعد پایین‌تر تبدیل می‌کنیم.
</div>

In [4]:
tf_idf, w2v, lables, links, df = JSON_to_clustering_arrays("./data/phase3/hamshahri.json")

In [5]:
features_dim = 50
random_state = 12
selected_tfidf = PCA(features_dim, random_state=random_state).fit_transform(tf_idf)

<div dir="rtl">
    <h3>
        بررسی پارامترهای k_means
    </h3>
    <p></p>
    تابع محاسبه k_means به این شکل است:
    <p></p>
    KMeans(n_clusters=n_clusters, random_state=random_state, n_init=10, max_iter=300)
    <p></p>
    n_cluster تعداد خوشه‌ها را مشخص می‌کند، random_state مقادیر اولیه شروع محاسبات رو تعیین می‌کند و عملا random_seed است. n_init تعداد دفعات انجام k_means را نشان می‌دهد که جواب نهایی بهترین جواب به دست آمده در این دفعات خواهد بود. max_iter تعداد بیشینه step ها را به ازای هر بار اجرای k_means مشخص می‌کند.
    <p></p>
    مهم‌ترین پارامتر n_cluster است. بنابراین ضمن ثابت نگه‌داشتن بقیه پارامترها، این مقدار را عوض کرده و بر اساس cost_function تعداد خوشه‌های مناسب به دست می‌آید.
    <p></p>
    cost_function یا inertia در واقع مجموع مجذور فاصله هر نقطه از مرکز خوشه‌ي خودش است.
    <p></p>
    بنابراین نمودار inertia را برحسب n_clusters رسم می‌کنیم.
</div>

In [ ]:
inertia = []
for i in range(2, 20):
    inertia.append(k_means.get_cost(selected_tfidf, random_state=random_state, n_clusters=i))
sns.lineplot(x=range(2, 20), y=inertia)

In [ ]:
print('TF-IDF')
_ = kelbow_visualizer(KMeans(random_state=12), selected_tfidf, k=(2,16), metric='silhouette');
print('W2V')
_ = kelbow_visualizer(KMeans(random_state=12), w2v, k=(2,16), metric='silhouette');